In [ ]:
import tensorflow as tf
from tensorflow import keras
from IPython.display import Audio

from music.midi import read
from music import stats
from visual import hist

tf.__version__

In [ ]:
midi_file = 'midi/track.mid'

track = read(midi_file)
track

In [ ]:
dist = stats.distribution(track.notes)

print(track)
print(len(stats.unique(track.notes)))
print(dist)

hist(dict(dist.most_common(10)))
